In [ ]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [ ]:
import gzip
import simplejson as json

In [ ]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [ ]:
scripts[:2]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

In [ ]:
len(scripts)

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [ ]:
practices[:2]

In [ ]:
len(practices)

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [ ]:
def mean(data):
    
    """Return the sample arithmetic mean of data"""
    
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    
    """Return sum of square deviations of sequence data"""
    
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def stddev(data, ddof=0):
    
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

def find_median(sorted_list):
    
    """Return a median of the sorted list and the indices 
    to calculate Q1 and Q3 after"""
    
    indices = []

    list_size = len(sorted_list)
    median = 0

    if list_size % 2 == 0:
        indices.append(int(list_size / 2) - 1)  # -1 because index starts from 0
        indices.append(int(list_size / 2))

        median = (sorted_list[indices[0]] + sorted_list[indices[1]]) / 2
        pass
    else:
        indices.append(int(list_size / 2))

        median = sorted_list[indices[0]]
        pass

    return median, indices
    pass

In [ ]:
def describe(key):
    """
    Calculates Statistic over a DataSet provided in a Dict SCRIPTS 
    
    Returns: 
    SUM, MEAN, STD, Q1, MEDIAN, Q3 
    """
    sorted_list = sorted([scripts[ilen][key] for ilen in range(len(scripts))])

    median, median_indices = find_median(sorted_list)
    Q1, Q1_indices = find_median(sorted_list[:median_indices[0]])
    Q3, Q3_indices = find_median(sorted_list[median_indices[-1] + 1:])
    
    total = sum(sorted_list)
    avg = mean(sorted_list)
    s = stddev(sorted_list)
    q25 = Q1
    med = median
    q75 = Q3
    
    return (total, avg, s, q25, med, q75)

In [ ]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [ ]:
grader.score.pw__summary_statistics(summary)

## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [ ]:
from collections import defaultdict

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [ ]:
def find_keys(data, var, count):
    count_dict = defaultdict(int)
    for ilen in range(len(data)): 
        ivar = data[ilen][var]
        count_dict[ivar] += data[ilen][count]
    return list(count_dict)

def find_keys2(data, var): 
    var_names = [str(ilen[var]) for ilen in data]
    return var_names 
    
def create_group(data, var_list, var_name): 
    groups = {name: [] for name in var_list}

    for script in data:
        groups[script[var_name]].append(script)
    return groups 

def find_sum_item(groups, item_name):
    group_sum = {}
    for i in groups:
        sum = 0
        for j in range(len(groups[i])):
            sum = sum + groups[i][j][item_name]
        group_sum.update({i: sum})
    return group_sum

def dict_find_max(d1):
    return [k for k,v in d1.items() if v == max(d1.values())][0]

In [ ]:
bnf_names2 = find_keys(scripts, 'bnf_name', 'items')
assert(len(bnf_names2) == 5619)

In [ ]:
bnf_names = find_keys2(scripts, 'bnf_name')
groups = create_group(scripts, bnf_names, 'bnf_name')

In [ ]:
groups_item = find_sum_item(groups, 'items')
max_item = [(dict_find_max(groups_item), groups_item[dict_find_max(groups_item)])]

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [ ]:
grader.score.pw__most_common_item(max_item)

**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [ ]:
def group_by_field(data, fields): 
    groups = {names: {} for names in fields} 
    for names in fields: 
        bnf_name = find_keys2(data, names)
        group = create_group(data, bnf_name, names)
        groups[names] = group
    return groups

In [ ]:
group_test = group_by_field(scripts, ('bnf_name','practice'))

In [ ]:
#group_test['bnf_name'].keys()

In [ ]:
groups2 = group_by_field(scripts, ('bnf_name',))
groups2 = find_sum_item(groups2['bnf_name'], 'items')
test_max_item = [(dict_find_max(groups2), groups2[dict_find_max(groups2)])]

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [ ]:
practices[0]

In [ ]:
len(practices)

In [ ]:
scripts[0]

In [ ]:
len(scripts)

In [ ]:
practice_postal = {}
for practice in practices:
    ## if code is already inside new DICT will assign postal code value or keep the previous one 
    ## first by alphabet 
    if practice['code'] in practice_postal:
        ## Will compae string as an increasing alphabetical order
        if practice_postal[practice['code']] < practice['post_code']:
            practice_postal[practice['code']] = practice_postal[practice['code']]
        else:
            practice_postal[practice['code']] = practice['post_code']

    else:
        practice_postal[practice['code']] = practice['post_code']
        
sorted_practice_postal = {k: v for k, v in sorted(practice_postal.items(), key=lambda item: item[1])}

In [ ]:
len(practice_postal)

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [ ]:
assert sorted_practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [ ]:
joined = scripts[:]
for script in joined:
    script['post_code'] = sorted_practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [ ]:
items_by_post2 = {}
for script in joined: 
    items_by_post2[script['post_code']] = items_by_post2.get(script['post_code'], 0) + script['items']
    
items_by_post2_sorted = {k: v for k, v in sorted(items_by_post2.items(), key=lambda item: item[0])}
postal_totals = [(str(key),value) for key, value in sorted(items_by_post2.items())]

In [ ]:
postal_totals = sorted(items_by_post2.items())[:100] 

grader.score.pw__postal_totals(postal_totals)

## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [ ]:
len(joined)

In [ ]:
len(scripts)

In [ ]:
total_items_bnf_post = {}
for script in joined:
    ## Probbaly has data that is empty 
    if (script['post_code'], script['bnf_name']) in total_items_bnf_post:
        total_items_bnf_post[(script['post_code'], script['bnf_name'])] += script['items']
    else: 
        total_items_bnf_post[(script['post_code'], script['bnf_name'])] = script['items']

In [ ]:
len(total_items_bnf_post)

In [ ]:
## THE length of the file changed??? 
## 
assert len(total_items_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [ ]:
for ikey in total_items_bnf_post.items():
    print(ikey)
    print(ikey[1])
    print(ikey[0])
    print(ikey[0][1])
    print(ikey[0][0])
    break

In [ ]:
total_dict = {} # empty
total_dict_items = {}
for ikey in total_items_bnf_post.items():

    ## if key post_code inside the dict - append the dict with values 
    if str(ikey[0][0]) in total_dict:
        #print('The same Key', str(ikey[0][0]))
        #ivar += ikey[1]
        total_dict[ikey[0][0]].update({ikey[0][1]:ikey[1]})
        
    else: 
        ## if the key does not exist yet we assign it 
        total_dict[str(ikey[0][0])] = {}
        #ivar = 0
    #total_dict_items[ikey[0][0]] = ivar

In [ ]:
def dict_sorted_max(d1):
    d3={v:k for k,v in sorted(d1.items())}
    return d3[max(d3)]

In [ ]:
dict_sorted_max(total_dict['SK11 6JL'])

In [ ]:
sum(total_dict['SK11 6JL'].values())

In [ ]:
items_by_region = []
for key in total_dict: 
    total_items = sum(total_dict[str(key)].values())
    name_bnf = dict_sorted_max(total_dict[str(key)])
    print(name_bnf)
    print(total_dict[key][name_bnf])
    items_by_region.append((str(key), name_bnf, total_dict[key][name_bnf]/total_items))

In [ ]:
len(items_by_region)

In [ ]:
assert len(total_dict_items) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [ ]:
max_item_by_post = sorted(items_by_region, key=lambda post_code: post_code[0])

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [ ]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [ ]:
grader.score.pw__items_by_region(max_item_by_post [:100])

In [ ]:
### Someone else code as an example 

In [ ]:
joined = scripts[:]
for script in joined:
    script['post_code'] = sorted_practice_postal[script['practice']]
    
# creating a post_code_list
post_code_list = []
for script in joined:
    post_code_list.append(script['post_code'])
post_code_list = set(sorted(post_code_list))

In [ ]:
dict_new = {post_code: [] for post_code in post_code_list}

for script in joined:
    if len(dict_new[script['post_code']])== 2 and dict_new[script['post_code']][0] == script['bnf_name']:
        dict_new[script['post_code']][1] += script['items']
    else:
        dict_new[script['post_code']].append((script['bnf_name'], script['items']))

In [ ]:
list_new = []
for key in dict_new.keys():
    for i in range(len(dict_new[key])):
        list_new.append({'post_code': key, 'bnf_name': dict_new[key][i][0], 
                         'total':dict_new[key][i][1]}) 

total_items_by_bnf_post = {(dict_info['post_code'], 
                            dict_info['bnf_name']): dict_info['total'] for dict_info in list_new}

total_items_by_post = {}
for dict_info in list_new:
    if dict_info['post_code'] in total_items_by_post.keys():
        total_items_by_post[dict_info['post_code']] += dict_info['total']
    else:
        total_items_by_post[dict_info['post_code']] = dict_info['total']

# generated a list of tuples with the codes, bnf_name and item total proportion
max_item_by_post = []
for key in dict_new.keys():
    max_item_by_post.append((key, (max(dict_new[key], key=lambda x:x[1]))[0],
                             float((max(dict_new[key], 
                                        key=lambda x:x[1]))[1])/total_items_by_post[key]))

# I sorted the list at this point
max_item_by_post = sorted(max_item_by_post, key=lambda post_code: post_code[0])

# slicing the list
items_by_region = max_item_by_post[:100]

In [ ]:
grader.score.pw__items_by_region(items_by_region[:100])

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*